# HCF Function

In [ ]:
import json
import random
import constraint
from techniques.PaDMTP import padmtp_algo
from techniques.RandomTesting import random_testing
from testcodes.hcf import compute_hcf as hcf
from techniques.AdaptiveRandomTesting import adaptive_random_testing

### PaDMTP

In [ ]:
problem = constraint.Problem()

problem.addVariable('x', range(1, 16))
problem.addVariable('y', range(1, 16))

problem.addConstraint(hcf, ['x', 'y'])

solutions = problem.getSolutions()

length = len(solutions)

In [ ]:
cleaned_solutions = []

for index, solution in enumerate(solutions):
    cleaned_solutions.append(solutions[index])
    
hcf_length = len(cleaned_solutions)

print("\nPossible Solutions Dataset for HCF Code: \n(x,y) ∈ {", end="")
for index, solution in enumerate(cleaned_solutions):
    if index == hcf_length - 1:
        print("({},{})".format(solution['x'], solution['y']), end="")
    else:
        print("({},{}),".format(solution['x'], solution['y']), end="")
print("}\n")

In [ ]:
print(f"HCF - PaDMTP - Test Cases: {hcf_length}")

In [ ]:
mutPyObj = 'mut.py --target testcodes.hcf --unit-test mutations.test_hcf_padmt -m'

padmtp_rslt = padmtp_algo(hcf, cleaned_solutions, mutPyObj)

In [ ]:
padmtp_rslt['PaDMTP_Overhead']

In [ ]:
padmtp_rslt['PaDMTP_DataFrame']

In [ ]:
PaDMTP_MT_Scores = []

for index, row in padmtp_rslt['PaDMTP_DataFrame'].iterrows():
    val = row['MT Score']
    val = val.replace("%", "")
    val = float(val) / 100
    val = round(val, 3)
    
    PaDMTP_MT_Scores.append(val)

### Random Testing (RT)

In [ ]:
# Generating random inputs
rt_test_cases = []
for i in range(hcf_length):
    x = random.randint(1, 101)
    y = random.randint(1, 101)
    
    rt_test_cases.append((x, y))

In [ ]:
print(f"HCF - Random Testing - Test Cases: {len(rt_test_cases)}")

In [ ]:
# Saving RT Test Cases to JSON File

jsonString = json.dumps(rt_test_cases)
jsonFile = open("./temp/RT_Test_Cases.json", 'w')
jsonFile.write(jsonString)
jsonFile.close()

In [ ]:
pytest_cmd = 'pytest techniques/rt_hcf.py'
mutpy_cmd = 'mut.py --target testcodes.hcf --unit-test mutations.test_hcf_rt -m'

rt_rslt = random_testing(pytest_cmd, mutpy_cmd, rt_test_cases)

In [ ]:
rt_rslt['RT_Overhead']

In [ ]:
rt_rslt['RT_DataFrame']

In [ ]:
RT_MT_Scores = []

for index, row in rt_rslt['RT_DataFrame'].iterrows():
    val = row['MT Score']
    val = val.replace("%", "")
    val = float(val) / 100
    val = round(val, 3)
    
    RT_MT_Scores.append(val)

### Adaptive Random Testing (ART)

In [ ]:
# Generating random inputs
art_test_cases = []
for i in range(hcf_length):
    x = random.randint(-100, 101)
    y = random.randint(-100, 101)
    
    art_test_cases.append([x, y])

In [ ]:
print(f"HCF - Adaptive Random Testing - Test Cases: {len(art_test_cases)}")

In [ ]:
mutpy_cmd = 'mut.py --target testcodes.hcf --unit-test mutations.test_hcf_art -m'

art_rslt = adaptive_random_testing(hcf, mutpy_cmd, art_test_cases, 'HCF')

In [ ]:
art_rslt['ART_Overhead']

In [ ]:
art_rslt['ART_DataFrame']

In [ ]:
ART_MT_Scores = []

for index, row in art_rslt['ART_DataFrame'].iterrows():
    val = row['MT Score']
    val = val.replace("%", "")
    val = float(val) / 100
    val = round(val, 3)
    
    ART_MT_Scores.append(val)

### Pairwise T-Test

In [ ]:
import numpy as np
from scipy.stats import ttest_rel, t
from tabulate import tabulate

# Data
PaDMTP = np.array(PaDMTP_MT_Scores) 
RT = np.array(RT_MT_Scores)
ART = np.array(ART_MT_Scores)

# Calculate paired differences
differences_PaDMTP_RT = np.subtract(PaDMTP, RT)
differences_PaDMTP_ART = np.subtract(PaDMTP, ART)

# Perform t-test: PaDMTP vs RT
t_statistic_PaDMTP_RT, p_value_PaDMTP_RT = ttest_rel(PaDMTP, RT)
t_critical_one_tail_PaDMTP_RT = t.ppf(0.05, len(PaDMTP) - 1)
t_critical_two_tail_PaDMTP_RT = t.ppf(0.025, len(PaDMTP) - 1)

# Perform t-test: PaDMTP vs ART
t_statistic_PaDMTP_ART, p_value_PaDMTP_ART = ttest_rel(PaDMTP, ART)
t_critical_one_tail_PaDMTP_ART = t.ppf(0.05, len(PaDMTP) - 1)
t_critical_two_tail_PaDMTP_ART = t.ppf(0.025, len(PaDMTP) - 1)

# Create the tables
table1 = [
    ["Test", "Mean", "Variance", "Observations", "df", "t Stat"],
    ["PaDMTP vs RT", np.mean(differences_PaDMTP_RT), np.var(differences_PaDMTP_RT), len(PaDMTP), len(PaDMTP) - 1, t_statistic_PaDMTP_RT],
    ["PaDMTP vs ART", np.mean(differences_PaDMTP_ART), np.var(differences_PaDMTP_ART), len(PaDMTP), len(PaDMTP) - 1, t_statistic_PaDMTP_ART]
]

table2 = [
    ["Test", "P(T<=t) one-tail", "t Critical one-tail", "P(T<=t) two-tail", "t Critical two-tail"],
    ["PaDMTP vs RT", p_value_PaDMTP_RT / 2, t_critical_one_tail_PaDMTP_RT, p_value_PaDMTP_RT, t_critical_two_tail_PaDMTP_RT],
    ["PaDMTP vs ART", p_value_PaDMTP_ART / 2, t_critical_one_tail_PaDMTP_ART, p_value_PaDMTP_ART, t_critical_two_tail_PaDMTP_ART]
]

print("<==== HCF Code ====>")

# Print the tables
print("Table 1:")
print(tabulate(table1, headers="firstrow", tablefmt="grid"))
print("\nTable 2:")
print(tabulate(table2, headers="firstrow", tablefmt="grid"))


# Perform pairwise t-tests
t_stat_RT_PaDMT, p_value_RT_PaDMT = ttest_rel(RT, PaDMTP)
t_stat_ART_PaDMT, p_value_ART_PaDMT = ttest_rel(ART, PaDMTP)

# Print the t-statistics and p-values
print("\n\nRT vs PaDMT:")
print("T-statistic:", t_stat_RT_PaDMT)
print("P-value:", p_value_RT_PaDMT)
print("\nART vs PaDMT:")
print("T-statistic:", t_stat_ART_PaDMT)
print("P-value:", p_value_ART_PaDMT)